In [2]:
import sys
import logging

import http.client
import json

import pandas as pd

In [3]:
logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)

c_handler = logging.StreamHandler(sys.stdout)
c_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# c_format = logging.Formatter(f'%(levelname)-8s: %(message)s')
c_handler.setFormatter(c_format)

for handler in logger.handlers:
    logger.removeHandler(handler)
logger.addHandler(c_handler)

In [23]:
logger.debug('meow')
logger.info('aku adalah anak woof woof')

2020-06-14 11:50:35,881 - __main__ - DEBUG - meow
2020-06-14 11:50:35,881 - __main__ - INFO - aku adalah anak woof woof


In [27]:
# TYPE = 'train'
TYPE = 'dev'

# DATASET_NAME = f'{TYPE}-v2.0-translated_fixed'    # This is for SQuAD dataset
DATASET_NAME = f'tydiqa-goldp-v1.1-{TYPE}-indonesian_prepared'    # This is for TyDiQA dataset

DATASET_PATH = f'../data/interim/{DATASET_NAME}.json'
SAVE_PATH = f'../data/processed/{DATASET_NAME}_enhanced.json'

df_qa = pd.read_json(DATASET_PATH)
df_qa = df_qa
print(df_qa.shape)
print(df_qa)

(5702, 2)
                            title  \
33713        Ernest Douwes Dekker   
33714            Balai Kota Seoul   
33715  Perserikatan Bangsa-Bangsa   
33716               James Hepburn   
33717                 Jamie Vardy   
...                           ...   
39410      Sejarah tabel periodik   
39411      Bosnia dan Herzegovina   
39412        Kereta Api Indonesia   
39413                Pencak silat   
39414           Kaisar Tang Gaozu   

                                              paragraphs  
33713  [{'context': 'Ernest Douwes Dekker wafat dini ...  
33714  [{'context': 'Pada tanggal 18 Februari 2008, d...  
33715  [{'context': 'Sebagai tindak lanjut Atlantic C...  
33716  [{'context': 'Dia dipenjarakan di Puri Dragsho...  
33717  [{'context': 'Lahir di Sheffield, South Yorksh...  
...                                                  ...  
39410  [{'context': 'Sejarah tabel periodik mencermin...  
39411  [{'context': 'Bosnia merupakan salah satu nega...  
39412  [{'cont

In [10]:
def get_pos_tag(text):
    conn = http.client.HTTPConnection("10.181.131.244:5500")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

def get_ner(text):
    conn = http.client.HTTPConnection("10.181.131.244:10009")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

In [12]:
text = "Frédéric Chopin! adalah, (1907–1986) (1907-1986) anak-anak (children): gembala. Andi's sheep is an '03 R&B player; 奉獻 km² Jīdū °5 вера ʰp. Totalnya 10.000 rupiah u?yea u!yea u,yea u.yea kura-kura bolak-balik"
pos_tag = get_pos_tag(text)
ner = get_ner(text)

print(pos_tag)
print(ner)

{'postags': [[['Frederic', 'NNP'], ['Chopin', 'NNP'], ['!', 'PUN']], [['adalah', 'VBL'], [',', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['anak-anak', 'NNO'], ['(', 'PUN'], ['children', 'NNO'], [')', 'PUN'], [':', 'PUN'], ['gembala', 'NNO'], ['.', 'PUN']], [['Andi', 'NNP'], ["'", 'PUN'], ['s', 'NNP'], ['sheep', 'NNP'], ['is', 'PPO'], ['an', 'PUN'], ["'", 'PUN'], ['03', 'NUM'], ['R', 'NNP'], ['&', 'PUN'], ['B', 'NNP'], ['player', 'NNP'], [';', 'PUN'], ['Feng', 'NNP'], ['Xian', 'NNP'], ['km2', 'NNP'], ['Jidu', 'NNP'], ['deg5', 'NNP'], ['vera', 'NNP'], ['kp', 'NNP'], ['.', 'PUN']], [['Total', 'NNO'], ['nya', 'PRK'], ['10.000', 'NUM'], ['rupiah', 'NNO'], ['u', 'ADJ'], ['?', 'PUN']], [['yea', 'NUM'], ['u', 'NNP'], ['!', 'PUN']], [['yea', 'NNO'], ['u', 'PUN'], [',', 'PUN'], ['yea', 'NNP'], ['u.ye', 'NNP'], ['a', 'PPO'], ['kura-kura', 'NNO'], ['bolak-balik', 'VBI']]]}
{'entiti

In [13]:
SLEEP_TIME = 10

def save_wait_recall_api(payload, call_api_func, df, path):
    print(f'Problem when calling API.. Saving, then sleeping for {SLEEP_TIME} seconds...')
    df.to_json(path)
    time.sleep(SLEEP_TIME)
    return call_api_func(payload)

In [14]:
class LastIdxTracker():
    def __init__(self, last_topic_idx, last_content_idx):
        self.topic = last_topic_idx
        self.content = last_content_idx

In [15]:
def add_postag_and_ner(df_qa, taken_topic_idx, taken_context_idx, path, last_idx_tracker, logger=logger):
    qas = df_qa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
    context = df_qa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
#     logger.debug(f'{context}\n')

    try:
        context_postags = get_pos_tag(context)
    except:
        context_postags = save_wait_recall_api(context, get_pos_tag, df_qa, path)
                    
    try:
        context_ner = get_ner(context)
    except:
        context_ner = save_wait_recall_api(context, get_ner, df_qa, path)
        
    df_qa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_postags)
    df_qa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_ner)
    
    last_idx_tracker.topic = taken_topic_idx
    last_idx_tracker.content = taken_context_idx
    
    for question in qas:
        try:
            question_postags = get_pos_tag(question['question'])
        except:
            question_postags = save_wait_recall_api(question['question'], get_pos_tag, df_qa, path)

        try:
            question_ner = get_ner(question['question'])
        except:
            question_ner = save_wait_recall_api(question['question'], get_ner, df_qa, path)
        question.update(question_postags)
        question.update(question_ner)

In [29]:
last_idx_tracker = LastIdxTracker(0, 0)

In [38]:
print(last_idx_tracker.topic)
print(last_idx_tracker.content)

657
0


In [39]:
import time


start_time = time.time()
logger.debug(f'Save path is: {SAVE_PATH}')
for taken_topic_idx in range(last_idx_tracker.topic, df_qa.shape[0]):
    logger.debug(f'Topic: {taken_topic_idx}')
    for taken_context_idx in range(last_idx_tracker.content, len(df_qa.iloc[taken_topic_idx]['paragraphs'])):
        logger.debug(f'\t{taken_context_idx}')
        add_postag_and_ner(df_qa, taken_topic_idx, taken_context_idx, SAVE_PATH, last_idx_tracker)
#         logger.debug(df_qa.iloc[taken_topic_idx]['paragraphs'][taken_context_idx])
    last_idx_tracker.content = 0
#     if last_idx_tracker.topic==143: break
print(time.time() - start_time)

2020-06-14 12:13:59,851 - __main__ - DEBUG - Save path is: ../data/processed/tydiqa-goldp-v1.1-train-indonesian_prepared_enhanced.json
2020-06-14 12:13:59,867 - __main__ - DEBUG - Topic: 657
2020-06-14 12:13:59,867 - __main__ - DEBUG - 	0
2020-06-14 12:14:00,335 - __main__ - DEBUG - Topic: 658
2020-06-14 12:14:00,335 - __main__ - DEBUG - 	0
2020-06-14 12:14:00,720 - __main__ - DEBUG - Topic: 659
2020-06-14 12:14:00,720 - __main__ - DEBUG - 	0
2020-06-14 12:14:01,236 - __main__ - DEBUG - Topic: 660
2020-06-14 12:14:01,236 - __main__ - DEBUG - 	0
2020-06-14 12:14:01,752 - __main__ - DEBUG - Topic: 661
2020-06-14 12:14:01,752 - __main__ - DEBUG - 	0
2020-06-14 12:14:02,191 - __main__ - DEBUG - Topic: 662
2020-06-14 12:14:02,191 - __main__ - DEBUG - 	0
2020-06-14 12:14:02,593 - __main__ - DEBUG - Topic: 663
2020-06-14 12:14:02,593 - __main__ - DEBUG - 	0
2020-06-14 12:14:02,987 - __main__ - DEBUG - Topic: 664
2020-06-14 12:14:02,994 - __main__ - DEBUG - 	0
2020-06-14 12:14:03,458 - __main_

In [40]:
df_qa.iloc[taken_topic_idx]['paragraphs'][-1]

{'context': 'Kaisar Tang Gaozu (Hanzi: 唐高祖, 566-26 Juni 635) atau yang nama aslinya Li Yuan (李渊) adalah pendiri dan kaisar pertama Dinasti Tang yang memerintah sejak 618 hingga 626. Ia tadinya adalah seorang gubernur yang mengepalai wilayah yang sekarang menjadi Provinsi Shanxi. Ketika Dinasti Sui dilanda huru-hara yang berujung pada terpecahnya Tiongkok menjadi negara-negara bagian yang dikuasai para pemimpin pemberontak dan mantan pejabat Sui, Li Yuan juga melakukan hal yang sama, atas saran putra keduanya, Li Shimin, ia memberontak dan mengangkat cucu Kaisar Yang dari Sui, Yang You sebagai kaisar boneka dengan gelar Kaisar Gong dan ia sendiri sebagai walinya. Sebagian besar masa pemerintahannya dipakai untuk menaklukan pesaing-pesaingnya dalam usaha mempersatukan negara. Ia meneruskan kebijakan Kaisar Wen dari Sui, sang pendiri Dinasti Sui dan membatalkan kebijakan represif Kaisar Yang. Ia terkenal akan kemurahhatiannya pada sesama sehingga menarik banyak orang-orang berbakat bekerj

In [24]:
# df_qa = df_qa[df_qa.index < 9]
# df_qa.iloc[-1]['paragraphs'][-1]

'../data/processed/tydiqa-goldp-v1.1-dev-indonesian_prepared_enhanced.json'

In [41]:
print(f'Saving to: {SAVE_PATH}')
df_qa.to_json(SAVE_PATH)

Saving to: ../data/processed/tydiqa-goldp-v1.1-train-indonesian_prepared_enhanced.json


In [11]:
article = '''Image fusion merupakan sebuah proses penggabungan beberapa citra masukan menjadi sebuah citra baru yang memiliki deskripsi paling akurat terhadap objek atau kejadian yang terjadi di dalam citra tersebut. Tujuan dari proses itu sendiri adalah untuk menghasilkan sebuah citra baru yang lebih informatif dan cocok digunakan untuk proses pengolahan citra lainnya. Selain itu, proses ini juga dapat memperkecil jumlah data yang harus ditransmisikan di dalam jaringan. Terdapat empat kategori dalam melakukan image fusion, yaitu multiview fusion, multimodal fusion, multitemporal fusion, dan multifocus fusion.'''.replace('\n', ' ')
print(get_pos_tag(article)['postags'])
print()
print(get_ner(article)['entities'])

[[['Image', 'NNO'], ['fusion', 'ADJ'], ['merupakan', 'VBL'], ['sebuah', 'NUM'], ['proses', 'NNO'], ['penggabungan', 'NNO'], ['beberapa', 'KUA'], ['citra', 'NNO'], ['masukan', 'NNO'], ['menjadi', 'VBI'], ['sebuah', 'NUM'], ['citra', 'NNO'], ['baru', 'ADJ'], ['yang', 'PRR'], ['memiliki', 'VBT'], ['deskripsi', 'NNO'], ['paling', 'ADV'], ['akurat', 'ADJ'], ['terhadap', 'PPO'], ['objek', 'NNO'], ['atau', 'CCN'], ['kejadian', 'NNO'], ['yang', 'PRR'], ['terjadi', 'VBP'], ['di', 'PPO'], ['dalam', 'PPO'], ['citra', 'NNO'], ['tersebut', 'ART'], ['.', 'PUN']], [['Tujuan', 'NNO'], ['dari', 'PPO'], ['proses', 'NNO'], ['itu', 'ART'], ['sendiri', 'ADJ'], ['adalah', 'VBL'], ['untuk', 'PPO'], ['menghasilkan', 'VBT'], ['sebuah', 'NUM'], ['citra', 'NNO'], ['baru', 'ADJ'], ['yang', 'PRR'], ['lebih', 'ADV'], ['informatif', 'ADJ'], ['dan', 'CCN'], ['cocok', 'ADJ'], ['digunakan', 'VBP'], ['untuk', 'PPO'], ['proses', 'NNO'], ['pengolahan', 'NNO'], ['citra', 'NNO'], ['lainnya', 'ADJ'], ['.', 'PUN']], [['Selain